In [1]:
from pathlib import Path
import pandas as pd


In [56]:
evaluation_dir = Path("../results/classification/evaluation_ensemble/ensemble_evaluation")

data = pd.DataFrame()
for idx, csv_path in enumerate(evaluation_dir.glob('./*/*.csv')):
    if idx == 0:
        data = pd.read_csv(str(csv_path))
        #data['dataset'] = csv_path.stem
    else:
        data_new = pd.read_csv(str(csv_path))
        #data_new['dataset'] = csv_path.stem
        data = data.append(data_new)

In [57]:
data.head()

,Unnamed: 0,step,model,dataset,errors,r_times,memories
0,0,1000,Hoeffding Tree,"RBF(10,0.0001)",0.981982,4.684223,0.187305
1,1,2000,Hoeffding Tree,"RBF(10,0.0001)",0.987494,17.856928,0.187305
2,2,3000,Hoeffding Tree,"RBF(10,0.0001)",0.989330,32.938135,0.187305
3,3,4000,Hoeffding Tree,"RBF(10,0.0001)",0.990498,44.532113,0.187305
4,4,5000,Hoeffding Tree,"RBF(10,0.0001)",0.990798,57.573747,0.187305


In [58]:
datasets = list(data['dataset'].unique())
models = list(data['model'].unique())

new_data = {}
new_data['dataset'] = []
for model in models:
    new_data['dataset'] = []
    new_data[f'{model}__Time'] = []
    new_data[f'{model}__Accuracy'] = []
    new_data[f'{model}__std'] = []
    new_data[f'{model}__Memory'] = []
    new_data[f'{model}__Rank'] = []


for dataset in datasets:
    print(dataset)
    dataset_data = data[data['dataset']==dataset]
    max_step = dataset_data['step'].max()
    ranks = dataset_data[(dataset_data['step'] == max_step)][['model','errors']]
    ranks['rank'] = ranks['errors'].rank(method='max', ascending=False)

    new_data['dataset'].append(dataset)
    for model in models:
        if list(dataset_data[(dataset_data['model']==model) & (dataset_data['step'] == max_step)]['r_times']) != []:
            #print(dataset_data[(dataset_data['model']==model) & (dataset_data['step'] == max_step)]['r_times'])
            time = "%.3f" % float(dataset_data[(dataset_data['model']==model) & (dataset_data['step'] == max_step)]['r_times'])
            accuracy = "%.4f" % float(dataset_data[(dataset_data['model']==model) & (dataset_data['step'] == max_step)]['errors'])
            std = "%.2f" % float(dataset_data[(dataset_data['model']==model)]['errors'].std())
            memory = "%.3f" % float(dataset_data[(dataset_data['model']==model)]['memories'].mean())
            new_data[f'{model}__Time'].append(time)
            new_data[f'{model}__Accuracy'].append(accuracy)
            new_data[f'{model}__std'].append(std)
            new_data[f'{model}__Memory'].append(memory)
            new_data[f'{model}__Rank'].append(float(ranks[(ranks['model']==model)]['rank']))
        else :
            new_data[f'{model}__Time'].append(0)
            new_data[f'{model}__Accuracy'].append(0)
            new_data[f'{model}__std'].append(0)
            new_data[f'{model}__Memory'].append(0)
            new_data[f'{model}__Rank'].append(99)

RBF(10,0.0001)
RBF(50,0.0001)
RBF(50,0.001)
SEA(50)
Hyperplane(50,0.001)
Hyperplane(50, 0.0001)
Agrawal(50,000)
SINE()
Covtype
LED()
SEA(50,000)
RBF(10,0.001)
Elec
Agrawal(50)


In [59]:
test = pd.DataFrame(new_data)

In [62]:
test.describe()


,Hoeffding Tree__Rank,Bagging__Rank,ARF__Rank,EvoAutoML Bagging Oldest__Rank,Logistic Regression__Rank,KNN__Rank,SRPC__Rank,Leveraging Bagging__Rank,EvoAutoML Bagging Best__Rank,HAT__Rank,GaussianNB__Rank
count,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000
mean,14.000000,12.142857,11.357143,10.642857,14.500000,13.571429,20.357143,9.857143,9.928571,12.071429,14.928571
std,24.491757,25.050279,25.391116,25.512010,24.613161,24.923620,33.401076,25.722953,25.658899,25.113479,24.332518
min,6.000000,3.000000,1.000000,1.000000,1.000000,1.000000,3.000000,1.000000,2.000000,2.000000,2.000000
25%,6.250000,4.000000,3.250000,2.250000,6.000000,2.750000,5.250000,1.250000,2.000000,3.500000,8.250000
50%,8.000000,6.500000,4.000000,3.500000,11.000000,9.000000,8.500000,2.500000,3.000000,6.000000,10.000000
75%,8.750000,7.000000,6.750000,5.750000,11.000000,10.000000,9.750000,5.000000,4.000000,7.750000,10.000000
max,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000
